# Questions for Board Meeting
### MVP 
1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
7. Which lessons are least accessed?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

### If there's time
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# DBSCAN import
from sklearn.cluster import DBSCAN

# Scaler import
from sklearn.preprocessing import MinMaxScaler

from env import host, user, password

## Aquire

In [2]:
url = f'mysql+pymysql://{user}:{password}@{host}/curriculum_logs'

In [3]:
sql_query = query = '''
SELECT date, time, path, user_id, cohort_id, program_id, ip, name, slack, start_date, end_date, created_at, updated_at
FROM logs
JOIN cohorts on logs.cohort_id = cohorts.id
'''

In [4]:
if os.path.isfile('logs.csv'):

    # If csv file exists, read in data from csv file.
    df = pd.read_csv('logs.csv', index_col=0)

else:

    # Read fresh data from db into a DataFrame.
    df = pd.read_sql(sql_query, url)
    
    # Write DataFrame to a csv
    df.to_csv('logs.csv')

In [5]:
df.head()

,date,time,path,user_id,cohort_id,program_id,ip,name,slack,start_date,end_date,created_at,updated_at
0,2018-01-26,09:55:03,/,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26
1,2018-01-26,09:56:02,java-ii,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,2,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10


In [6]:
df.shape

(847330, 13)

## Prepare

In [7]:
# let's restart this
conditions = [df.program_id == 1, df.program_id == 2, df.program_id == 3, df.program_id == 4]
result = ['web_dev','web_dev','data_science','web_dev']
df['program'] = np.select(conditions, result)
df.head()

,date,time,path,user_id,cohort_id,program_id,ip,name,slack,start_date,end_date,created_at,updated_at,program
0,2018-01-26,09:55:03,/,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
1,2018-01-26,09:56:02,java-ii,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,2,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,web_dev


## 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
- Break down lessons in path by filtering by defining a path that belongs to program and then lesson
    - groupby path and count
    - assign path to label. if path == ??? then column label is lesson
    
## Find Lessons for Data Scince

In [8]:
# subset for data science
df_ds = df[df.program=='data_science']
df_ds.head()

,date,time,path,user_id,cohort_id,program_id,ip,name,slack,start_date,end_date,created_at,updated_at,program
300225,2019-08-20,09:39:58,/,466,34.0,3,97.105.19.58,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,data_science
300226,2019-08-20,09:39:59,/,467,34.0,3,97.105.19.58,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,data_science
300227,2019-08-20,09:39:59,/,468,34.0,3,97.105.19.58,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,data_science
300228,2019-08-20,09:40:02,/,469,34.0,3,97.105.19.58,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,data_science
300229,2019-08-20,09:40:08,/,470,34.0,3,97.105.19.58,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,data_science


In [9]:
# group df by path and create column for count of rows for each grouped path
df_ds_lesson_count = df_ds.groupby(by=['path'])[['user_id']].agg('count').sort_values('user_id', ascending=False)[1:60].reset_index()
df_ds_lesson_count.head()

,path,user_id
0,search/search_index.json,2203
1,classification/overview,1785
2,1-fundamentals/modern-data-scientist.jpg,1655
3,1-fundamentals/AI-ML-DL-timeline.jpg,1651
4,1-fundamentals/1.1-intro-to-data-science,1633


In [10]:
# # create lesson column
# df['lesson'] = np.where(df.path.str.contains('classification'),'classification',
#                         np.where(df.path.str.contains('sql'), 'sql',
#                                 np.where(df.path.str.contains('fundamentals'), 'fundamentals',
#                                         np.where(df.path.str.contains('regression'), 'regression',
#                                                 np.where(df.path.str.contains('python'), 'python',
#                                                         np.where(df.path.str.contains('stats'), 'stats', 
#                                                                  np.where(df.path.str.contains('anomaly'), 'anomaly',
#                                                                           np.where(df.path.str.contains('clustering'), 'clustering',
#                                                                                    np.where(df.path.str.contains('nlp'), 'nlp',
#                                                                                             np.where(df.path.str.contains('timeseries'), 'time_series',
#                                                                                                      np.where(df.path.str.contains('distributed-ml'), 'distributed_ml''',
#                                                                                                               np.where(df.path.str.contains('storytelling'), 'storytelling',
#                                                                                                                        np.where(df.path.str.contains('appendix'), 'appendix',
#                                                                                                                                 np.where(df.path.str.contains('advanced-topics'), 'advanced-topics',
#                                                                                                                                          np.where(df.path.str.contains('capstones'), 'capstones',
#                                                                                                                                                   'pending')))))))))))))))
# df.lesson.unique()

In [19]:
# create lesson column
df_ds['lesson'] = np.where(df_ds.path.str.contains('appendix'), 'appendix',
np.where(df_ds.path.str.contains('search'), 'search',
np.where(df_ds.path.str.contains('classification'),'classification',
np.where(df_ds.path.str.contains('sql'), 'sql',
np.where(df_ds.path.str.contains('fundamentals'), 'fundamentals',
np.where(df_ds.path.str.contains('regression'), 'regression',
np.where(df_ds.path.str.contains('python'), 'python',
np.where(df_ds.path.str.contains('stats'), 'stats', 
np.where(df_ds.path.str.contains('anomaly'), 'anomaly',
np.where(df_ds.path.str.contains('clustering'), 'clustering',
np.where(df_ds.path.str.contains('nlp'), 'nlp',
np.where(df_ds.path.str.contains('timeseries'), 'time_series',
np.where(df_ds.path.str.contains('distributed-ml'), 'distributed_ml''',
np.where(df_ds.path.str.contains('storytelling'), 'storytelling',

np.where(df_ds.path.str.contains('advanced-topics'), 'advanced-topics',
np.where(df_ds.path.str.contains('capstones'), 'capstones',
'pending'))))))))))))))))
df_ds.lesson.unique()

array(['pending', 'sql', 'storytelling', 'appendix', 'fundamentals',
       'search', 'advanced-topics', 'regression', 'anomaly', 'nlp',
       'classification', 'clustering', 'time_series', 'stats', 'python',
       'distributed_ml', 'capstones'], dtype=object)

In [20]:
df.head()

,date,time,path,user_id,cohort_id,program_id,ip,name,slack,start_date,end_date,created_at,updated_at,program
0,2018-01-26,09:55:03,/,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
1,2018-01-26,09:56:02,java-ii,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,2,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,web_dev


In [22]:
pending_lesson = df_ds[df_ds.lesson=='pending']
pending_lesson_ds = pending_lesson[pending_lesson.program=='data_science']
pending_lesson_ds_path = pending_lesson_ds.groupby('path')[['user_id']].agg('count').sort_values(by='user_id',ascending=False)
pending_lesson_ds_path.head(10)

,user_id
path,
/,8358
individual-project/individual-project,81
AI-ML-DL-timeline.jpg,58
modern-data-scientist.jpg,58
html-css,16
acquire,16
working-with-time-series-data,14
3-vocabulary.md,12
series,12


## Data Science Lessons (14 count): (taught to students )
- fundamentals
- sql
- capstones
- python
- regression'
- advanced-topics
- classification
- distributed-ml
- status
- clustering
- time_series
- anomaly
- nlp
- storytelling

## Data Science Not-Lessons (paths that are not curriculum)
- pending
- appendix

## Find Lessons for Web Dev

In [23]:
# subset for web_dev
df_webdev = df[df.program=='web_dev']
df_webdev.head()

,date,time,path,user_id,cohort_id,program_id,ip,name,slack,start_date,end_date,created_at,updated_at,program
0,2018-01-26,09:55:03,/,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
1,2018-01-26,09:56:02,java-ii,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,web_dev
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,2,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,web_dev


In [24]:
# group df by path and create column for count of rows for each grouped path
df_webdev_lesson_count = df_webdev.groupby(by=['path'])[['user_id']].agg('count').sort_values('user_id', ascending=False)[1:60].reset_index()
df_webdev_lesson_count.head(60)

,path,user_id
0,javascript-i,18193
1,toc,17580
2,search/search_index.json,15331
3,java-iii,13162
4,html-css,13111
5,java-ii,12173
6,spring,11877
7,jquery,11037
8,mysql,10602
9,java-i,10460


In [28]:
# created lesson column with values for web dev lessons
df_webdev['lesson'] = np.where(df_webdev.path.str.contains('search'),'search',
np.where(df_webdev.path.str.contains('index'),'index',
np.where(df_webdev.path.str.contains('javascript'),'javascript',
np.where(df_webdev.path.str.contains('toc'),'toc',
np.where(df_webdev.path.str.contains('java'),'java',
np.where(df_webdev.path.str.contains('html|css'),'html-css',
np.where(df_webdev.path.str.contains('spring'),'spring',
np.where(df_webdev.path.str.contains('jquery'),'jquery',
np.where(df_webdev.path.str.contains('mysql'),'mysql',
np.where(df_webdev.path.str.contains('capstone'),'capstone',
np.where(df_webdev.path.str.contains('array|syntax|object_oriented|polymorph|methods|collections|deployment'),'structure',
np.where(df_webdev.path.str.contains('php'),'php',
np.where(df_webdev.path.str.contains('larvel'),'larvel',
         
'pending')))))))))))))
                                                                                                                                                
df_webdev.lesson.unique()

array(['pending', 'java', 'structure', 'javascript', 'search', 'spring',
       'capstone', 'index', 'html-css', 'mysql', 'jquery', 'php', 'toc'],
      dtype=object)

In [31]:
pending_lesson = df_webdev[df_webdev.lesson=='pending']
pending_lesson_webdev = pending_lesson[pending_lesson.program=='web_dev']
pending_lesson_webdev_path = pending_lesson_ds.groupby('path')[['user_id']].agg('count').sort_values(by='user_id',ascending=False)
pending_lesson_webdev_path.head(10)

,user_id
path,
/,8358
individual-project/individual-project,81
AI-ML-DL-timeline.jpg,58
modern-data-scientist.jpg,58
html-css,16
acquire,16
working-with-time-series-data,14
3-vocabulary.md,12
series,12


## Web Dev Lessons:
- java
- structure
- javascript
- spring
- capstone
- html-css
- mysql
- jquery
- php
- toc

## See Counts for Data Science Lesson

In [36]:
# DS Lessons count
df_ds.groupby('lesson')[['user_id']].agg('count').sort_values(by='user_id', ascending=False)

,user_id
lesson,
fundamentals,16691
sql,13685
classification,11841
python,10492
pending,8940
regression,7471
stats,6663
anomaly,4332
clustering,4277


## Top 5 Popular Lessons for Data Science
- Fundamentals
- SQL
- Classfication
- Python
- Regression

## See Counts for Web Dev Lesson

In [38]:
# see counts for web dev lessons
df_webdev.groupby('lesson')[['user_id']].agg('count').sort_values(by='user_id', ascending=False)

,user_id
lesson,
javascript,158441
java,150598
html-css,98854
pending,94110
mysql,78868
spring,58178
jquery,54274
toc,17580
search,17384


## Top 5 Popular Lessons for Web Dev
- Javascript
- Java
- HTML-CSS
- MySQL
- Spring

# Code from Bonus: Identify users who are viewing both the web dev and data science curriculum 

In [ ]:
# find data science students that have logs for web deb pages
# subset df to data_science only
df_data_science = df[df.program_name=='data_science']
df_data_science.head()

In [ ]:
# find data science student endpoints that contain java or html
df_data_science.endpoint.str.contains(pat = 'html|java', case=False, regex=True).sum() # Too few to continue

In [ ]:
# make list of data science endpoints
ds_endpoints = df_data_science.endpoint.unique()
ds_endpoints = pd.Series(ds_endpoints)
ds_endpoints

In [ ]:
ds_endpoints.str.contains('java|html', case=False, regex=True)

In [ ]:
# find web dev students with endpoints of data science endpoints
df_web_dev = df[df.program_name=='web_dev']
df_web_dev.head()

In [ ]:
df_web_dev[df_web_dev.endpoint.isin(ds_endpoints)] # 234K observation of web dev have 